In [1]:
import os
import glob
import json
import PIL
from tqdm import tqdm
import time
from PIL import Image

In [2]:
# Deepfashion dataset path
men_dataset_path = '/data/Deepfashion-high/In-shop/img/MEN'
women_dataset_path = '/data/Deepfashion-high/In-shop/img/WOMEN'
list_description_inshop = '/data/Deepfashion-high/In-shop/Anno/list_description_inshop.json'
laundry_image_path = '/data/Deepfashion-high/In-shop/laundry_image'
list_bbox_inshop = '/data/Deepfashion-high/In-shop/Anno/list_bbox_inshop.txt'

In [3]:
men_category = ['Tees_Tanks', 'Pants', 'Shorts', 'Sweatshirts_Hoodies', 'Shirts_Polos', 'Sweaters', 'Jackets_Vests', 'Denim', 'Suiting']
women_category = ['Tees_Tanks', 'Blouses_Shirts', 'Dresses', 'Shorts', 'Sweaters', 'Skirts', 'Jackets_Coats', 'Pants', 'Rompers_Jumpsuits', 'Cardigans', 'Graphic_Tees', 'Sweatshirts_Hoodies', 'Denim', 'Leggings']

In [4]:
def get_item_laundry_label(image_item):
    with open(list_description_inshop) as f:
        data = json.load(f)
        for d in data:
            tmp = d['item']
            if tmp == image_item:
                des = d['description']
                if len(des) != 0:
                    return des[-2]

print(get_item_laundry_label('id_00000241'))

Machine wash cold


In [5]:
def get_laundry_category():
    with open(list_description_inshop) as f:
        data = json.load(f)

        laundry_category = []

        for d in data:
            tmp = d['description']
            if len(tmp) != 0:
                laundry_category.append(tmp[-2])
    
    return laundry_category

laundry_category = get_laundry_category()


In [6]:
laundry_category_count = {}

for label in laundry_category:
    if label not in laundry_category_count:
        laundry_category_count[label] = 1
    else:
        laundry_category_count[label] += 1

print(laundry_category_count)

print(laundry_category_count.keys())
print(len(laundry_category_count.keys()))

{'Hand wash cold': 4442, 'Machine wash cold': 2592, 'Hand wash cold, dry flat': 138, 'Dry clean': 535, 'Machine wash warm': 100, 'Hand wash cold, line dry': 53, 'Machine wash cold, line dry': 4, 'Hand wash, dry flat': 1, 'Machine wash cold, tumble dry low': 68, 'Machine wash cold, dry flat': 30, 'Hand wash cold, dry flat ': 11, 'Hand wash cold, hand dry': 1, 'Machine wash cold, dry flat ': 3, 'Dry clean only': 8, ' Dry clean ': 2, 'Measured from Medium': 3, '100% cotton': 2, ' Hand wash, flat dry ': 2, 'Machine wash cold, tumble dry low ': 21, 'Hand wash, line dry': 4, ' Dry clean only ': 1, 'Spot clean only': 1, 'Hand wash cold, tumble dry low ': 3, 'Dry Clean': 2, 'Machine wash, tumble dry  ': 1, 'Hand wash cold, tumble dry low': 2, ' Machine wash, tumble dry  ': 1, 'Dry clean ': 4, 'Machine wash warm, tumble dry low': 2, 'Machine wash, flat dry ': 2, 'Machine wash cold, tumble dry medium': 3, 'Wipe clean with damp cloth and soap': 1, 'Machine wash warm, line dry': 1, 'Machine wash w

In [7]:
# cat = ['Hand wash cold', 'Machine wash cold', 'Hand wash cold, dry flat', 'Dry clean', 'Machine wash warm', 'Hand wash cold, line dry', 'Machine wash cold, line dry', 'Hand wash, dry flat', 'Machine wash cold, tumble dry low', 'Machine wash cold, dry flat', 'Hand wash cold, dry flat ', 'Hand wash cold, hand dry', 'Machine wash cold, dry flat ', 'Dry clean only', ' Dry clean ', 'Measured from Medium', '100% cotton', ' Hand wash, flat dry ', 'Machine wash cold, tumble dry low ', 'Hand wash, line dry', ' Dry clean only ', 'Spot clean only', 'Hand wash cold, tumble dry low ', 'Dry Clean', 'Machine wash, tumble dry  ', 'Hand wash cold, tumble dry low', ' Machine wash, tumble dry  ', 'Dry clean ', 'Machine wash warm, tumble dry low', 'Machine wash, flat dry ', 'Machine wash cold, tumble dry medium', 'Wipe clean with damp cloth and soap', 'Machine wash warm, line dry', 'Machine wash warm, dry flat', 'Wipe clean with soap and damp cloth', 'Hand wash cold, line dry ', ' Machine wash warm, line dry ', ' Machine wash cold, dry flat ', ' Machine wash, line dry ', ' Machine wash, dry flat  ', ' Hand wash, flat dry  ', 'Machine wash, line dry', 'Hand wash, line dry  ', 'Machine wash Cold', 'Hand Wash Cold', 'Hand wash cold ', ' Hand wash cold, dry flat ', ' Hand wash, line dry ', 'Machine wash cold, tumble dry', 'Hand wash, line dry ', 'Machine wash cold, lay flat to dry ', 'Wipe clean with soap and  a damp cloth', ' Machine wash cold, line dry', 'Dry clean only ', 'Non dry clean', 'Professional leather cleaning only', 'Measured from Small']

# def create_dataset_split_structure(category):
#     for c in category:
#         if not os.path.exists(laundry_image_path + '/' +c):
#             os.makedirs(laundry_image_path + '/' +c)

# create_dataset_split_structure(cat)

In [8]:
# def get_gt_bbox(bbox_name, file_ptr):
#     with open(file_ptr) as bbox_file_ptr:
#         for line in bbox_file_ptr:
#             # print(line)
#             if bbox_name in line:
#                 x1 = int(line.split()[3])
#                 y1 = int(line.split()[4])
#                 x2 = int(line.split()[5])
#                 y2 = int(line.split()[6])
#                 bbox = [x1, y1, x2, y2]
#                 return bbox

# print(get_gt_bbox('img/WOMEN/Blouses_Shirts/id_00000001/02_1_front.jpg', list_bbox_inshop))

In [9]:
def calculate_bbox_and_save_img(image_path, image_save_name, bbox):
    x1 = bbox[0]
    y1 = bbox[1]
    x2 = bbox[2]
    y2 = bbox[3]
    img_read = Image.open(image_path)
    img_crop = img_read.crop((x1, y1, x2, y2))
    img_crop.save(image_save_name)

image_path = '/data/Deepfashion-high/In-shop/img/MEN/Tees_Tanks/id_00000045/01_1_front.jpg'
image_save_name = '/data/Deepfashion-high/In-shop/crop_test/1.jpg'


In [10]:
front_image_name = []
front_image_laundry_label = []

for cls in men_category:
    for f in tqdm(glob.glob(men_dataset_path + '/' + cls + '/**' + '/*.jpg')):
        time.sleep(0.1)
        # image_path: /data/Deepfashion-high/In-shop/img/MEN/Tees_Tanks/id_00000045/01_1_front.jpg
        image_path = f
        # print(image_path)
        image_name = image_path.split('/')[8].split('.')[0].split('_')[2] # image_name: front
        if image_name == 'front':
            # print(image_name)
            image_item = image_path.split('/')[7] # image_item: id_00000045
            laundry_label = get_item_laundry_label(image_item) # laundry_label: Dry clean
            if str(laundry_label) != 'None':
                # bbox_path_name = image_path.split('/')
                # bbox_name = bbox_path_name[4] + '/' + bbox_path_name[5] + '/' + bbox_path_name[6] + '/' + bbox_path_name[7] + '/' + bbox_path_name[8] 
                # # img/MEN/Tees_Tanks/id_00000045/01_1_front.jpg
                # bbox = get_gt_bbox(bbox_name, list_bbox_inshop) # [110, 36, 188, 142]
                image_save_name = laundry_image_path + '/' + laundry_label + '/' + image_item + '_' +  image_name + '_' + laundry_label + '.jpg'
                # calculate_bbox_and_save_img(image_path, image_save_name, bbox)
                img_read = Image.open(image_path)
                img_read.save(image_save_name)

        

100%|██████████| 39/39 [00:05<00:00,  6.84it/s]


In [12]:
# front_image_name = []
# front_image_laundry_label = []

for cls in women_category:
    for f in tqdm(glob.glob(women_dataset_path + '/' + cls + '/**' + '/*.jpg')):
        time.sleep(0.1)
        # image_path: /data/Deepfashion-high/In-shop/img/MEN/Tees_Tanks/id_00000045/01_1_front.jpg
        image_path = f
        # print(image_path)
        image_name = image_path.split('/')[8].split('.')[0].split('_')[2] # image_name: front
        if image_name == 'front':
            # print(image_name)
            image_item = image_path.split('/')[7] # image_item: id_00000045
            laundry_label = get_item_laundry_label(image_item) # laundry_label: Dry clean
            if str(laundry_label) != 'None':
                # bbox_path_name = image_path.split('/')
                # bbox_name = bbox_path_name[4] + '/' + bbox_path_name[5] + '/' + bbox_path_name[6] + '/' + bbox_path_name[7] + '/' + bbox_path_name[8] 
                # # img/MEN/Tees_Tanks/id_00000045/01_1_front.jpg
                # bbox = get_gt_bbox(bbox_name, list_bbox_inshop) # [110, 36, 188, 142]
                image_save_name = laundry_image_path + '/' + laundry_label + '/' + image_item + '_' +  image_name + '_' + laundry_label + '.jpg'
                # calculate_bbox_and_save_img(image_path, image_save_name, bbox)
                img_read = Image.open(image_path)
                img_read.save(image_save_name)



100%|██████████| 335/335 [00:48<00:00,  6.93it/s]


In [ ]:
drop out